In [32]:
import sys
import torch
from yaml import Loader
import numpy as np
import scipy.io

sys.path.append('../')
sys.path.append('../hifigan/')

In [33]:
from cube.networks.modules import ConvNorm, LinearNorm, PreNet, PostNet, Languasito2
from cube.io_utils.io_text import Text2FeatBlizzard
from cube.io_utils.io_cubegan import CubeganEncodings, CubeganCollate, CubeganDataset
from cube.networks.cubegan import Cubegan

In [34]:
text2feat = Text2FeatBlizzard('../data/phonemizer-blizzard')
rez = text2feat('C\'est un test simple avec le mot expatrier.')
for ii in range(len(rez['phones'])):
    phon = rez['phones'][ii]
    word = rez['words'][rez['phon2word'][ii]]
    print(phon, "\t", word)

§ 	 §
s 	 C
' 	 '
e^ 	 est
t 	 est
  	  
x~ 	 un
  	  
t 	 test
e^ 	 test
  	  
s 	 simple
e~ 	 simple
p 	 simple
l 	 simple
  	  
a 	 avec
v 	 avec
e^ 	 avec
k 	 avec
  	  
l 	 le
x^ 	 le
  	  
m 	 mot
o^ 	 mot
  	  
e^ 	 expatrier
k&s 	 expatrier
p 	 expatrier
a 	 expatrier
t 	 expatrier
r 	 expatrier
i&j 	 expatrier
e 	 expatrier
. 	 .
§ 	 §


In [35]:
# load cubegan
encodings = CubeganEncodings()
encodings.load('../data/cubegan-neb-fasttext.encodings')
import yaml

conf = yaml.load(open('../data/cubegan-neb-fasttext.yaml'), Loader)
model = Cubegan(encodings, conditioning=conf['conditioning'])
model.load('../data/cubegan-neb-fasttext.last')
model.eval()
enc = encodings
collate = CubeganCollate(enc, conditioning_type=conf['conditioning'])



fr


In [43]:
def synth(text:str):
    print('Start')
    rez={'meta':text2feat(text)}
    rez['meta']['speaker']='none'
    rez['pitch']=np.zeros((100))
    rez['mgc']=np.zeros((100,80))
    rez['meta']['words_left']=[]
    rez['meta']['words_right']=[]
    rez['meta']['frame2phon']=[0]*100
    with torch.no_grad():
        X = collate.collate_fn([rez])
        for key in X:
            if isinstance(X[key], torch.Tensor):
                X[key] = X[key].to(model.get_device())
        audio = model.inference(X)
        audio = audio.detach().cpu().numpy().squeeze()
        audio = np.asarray(audio * 32767, dtype=np.int16)
        scipy.io.wavfile.write('tmp.wav', 24000, audio)
    return audio

In [49]:
import IPython
audio=synth('Bonjour! Je suis un system artificialle.')
IPython.display.Audio("tmp.wav")

Start


In [57]:
audio=synth('Paris est la capitale de la France. \n\nDivisée en vingt arrondissements, elle est le chef-lieu de la région Île-de-France et le siège de la métropole du Grand Paris. \n\nElle est établie au centre du Bassin parisien, sur une boucle de la Seine, entre les confluents avec la Marne et l\'Oise.')
IPython.display.Audio("tmp.wav")

Start
